In [ ]:
import h5py
import pandas as pd 
import scanpy as sc
import anndata
import ipywidgets as widgets
import numpy as np
import wot
import matplotlib.pyplot as plt

In [ ]:
#in geo
adata=sc.read_loom("Genescore_foropt_fetalheart1.loom")
#

In [ ]:
adata_cell_details=pd.read_csv('Cell_details.csv')
adata_cell_details.index=adata_cell_details['index']


In [ ]:
adata.obs['Clusters1']=adata_cell_details['Clusters1']
adata.obs['index']=adata_cell_details['index']

In [ ]:
adata.obs['Day']=adata.obs['orig_ident']
adata.obs['Day']=adata.obs['Day'].str.replace('F19','3')
adata.obs['Day']=adata.obs['Day'].str.replace('F8','2')
adata.obs['Day']=adata.obs['Day'].str.replace('F6','1')
adata.obs['Day']=adata.obs['Day'].astype(int)


In [ ]:
coord_df=pd.read_csv('coords_df.csv')
coord_df.columns=['index','x','y']
coord_df.index=coord_df['index']                       
del coord_df['index']


In [ ]:
for_optimal_transport_obs=pd.read_csv('adata_obs_for_peaks.csv')
for_optimal_transport_obs.index=for_optimal_transport_obs['index']


In [ ]:
adata.obs['cell_growth_rate']=for_optimal_transport_obs['cell_growth_rate']
adata.obs['day']=for_optimal_transport_obs['day']


In [ ]:
days_df=pd.DataFrame(adata.obs['Day'])

In [ ]:
unique_days = adata.obs['Day'].unique()

In [ ]:
adata_variable=adata[:,adata.var[adata.var['Selected']==1].index]


In [ ]:
ot_model = wot.ot.OTModel(adata_variable, epsilon = 0.05, lambda1 = 1, lambda2 = 50, growth_iters = 3) 


In [ ]:
ot_model.compute_all_transport_maps(tmap_out='tmaps/GeneScore_OT')


In [ ]:
tmap_model = wot.tmap.TransportMapModel.from_directory('tmaps/GeneScore_OT')


In [ ]:
cell_sets_path='major_cell_set.gmt'
cell_sets = wot.io.read_sets(cell_sets_path, as_dict=True)


In [ ]:
start_populations = tmap_model.population_from_cell_sets(cell_sets, at_time=1)
end_populations = tmap_model.population_from_cell_sets(cell_sets, at_time=3)
transition_table = tmap_model.transition_table(start_populations, end_populations)
#
trans_tab_df=pd.DataFrame(transition_table.X)
trans_tab_df=trans_tab_df.round(2)
trans_tab_df.columns=transition_table.var_names
trans_tab_df.index=transition_table.obs_names
trans_tab_df=trans_tab_df.drop('Other')
#
#print(trans_tab_df.tail())

In [ ]:
start_populations = tmap_model.population_from_cell_sets(cell_sets, at_time=1)
end_populations = tmap_model.population_from_cell_sets(cell_sets, at_time=2)
transition_table = tmap_model.transition_table(start_populations, end_populations)
trans_tab_df1=pd.DataFrame(transition_table.X)
trans_tab_df1=trans_tab_df1.round(2)
trans_tab_df1.columns=transition_table.var_names
trans_tab_df1.index=transition_table.obs_names
trans_tab_df1=trans_tab_df1.drop('Other')


In [ ]:
start_populations = tmap_model.population_from_cell_sets(cell_sets, at_time=2)
end_populations = tmap_model.population_from_cell_sets(cell_sets, at_time=3)
transition_table = tmap_model.transition_table(start_populations, end_populations)

trans_tab_df2=pd.DataFrame(transition_table.X)
trans_tab_df2=trans_tab_df2.round(2)
trans_tab_df2.columns=transition_table.var_names
trans_tab_df2.index=transition_table.obs_names


In [ ]:
ttable=trans_tab_df+trans_tab_df1+trans_tab_df2
ttable[ttable <= 0.01] = 0

In [ ]:
import seaborn
import matplotlib
matplotlib.rcParams['pdf.fonttype']=42
plt.rcParams["figure.figsize"]=10,10
plt.rcParams['axes.xmargin'] = 0

font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 6}

matplotlib.rc('font', **font)
a=seaborn.heatmap(ttable.loc[['Endocaridum', 'Capillary_endothelium', 'Arterial_endothelium',
       'Myocardium', 'Ventricular_Cardiomyocytes','Atrial_Cardiomyocytes', 'Early_OFT_SMC', 'Vasculatur_development','vSMC','Pericytes',
       'Cardiac_fibroblast_progenitors', 'Early_Cardiac_fibroblast',
       'Cardiac_Fibroblast', 'Endocardial_cushion'],
        [ 'Endocaridum', 'Capillary_endothelium', 'Arterial_endothelium','Venal_endothelium','Endocaridum_unidentifed',
        'Ventricular_Cardiomyocytes','Atrial_Cardiomyocytes', 'Vasculatur_development','vSMC','Pericytes',
        'Early_Cardiac_fibroblast',
       'Cardiac_Fibroblast','Endocardial_cushion_late','Neural_Crest','Undifferentiated_epicardium']],cmap="hot_r", linewidths=0.01, linecolor='black')
plt.grid(False)
plt.ylabel('Parent Cell type')
plt.xlabel('Child Cell type')
plt.show()